# Create a linear regression model

- build a dataset class
- build a model class: init / fit / predict
- new function: np.nonzero / np.random.choice / np.arange / np.random.shuffle
- np.random.rand vs np.random.randn vs np.random.uniform

In [1]:
import numpy as np

In [2]:
class Dataset:
    def __init__(self, N):
        self.N = N
        self.X = np.random.rand(N, 2+1) * 2 - 1
        self.X[:, 2] = 1
        
        ps = np.random.rand(2, 2) * 2 - 1
        slope = (ps[1,1]-ps[0,1])/(ps[1,0]-ps[0,0])
        intercept = ps[0,1]-slope*ps[0,0]
        self.y = 2 * (self.X[:,1] > slope * self.X[:,0]\
                      + intercept) - 1        

In [76]:
class LinearRegression():
    def __init__(self):
        self.w = 0
    
    def fit(self, X, Y):
        self.w = np.linalg.solve(X.T@X, X.T@Y)
    
    def predict(self, X):
        return np.sign(X@self.w)
    
    def get_weights():
        return self.w

In [4]:
total_Ein = 0

for i in range(1000):
    ds = Dataset(100)
    lr = LinearRegression()
    lr.fit(ds.X, ds.y)
    total_Ein += np.sum(lr.predict(ds.X) != ds.y) / ds.N

total_Ein / 1000

0.03817000000000009

# Create a percepton model

In [5]:
class PLA:
    def __init__(self):
        self.w = 0
    
    def fit(self, X, y, initial_w = None):
        self.w = np.zeros((X.shape[1], 1))
        if initial_w is not None:
            self.w = initial_w
        self.iters = 0
        
        while True:
            yhat = np.sign(X @ self.w)
            misclassified = np.nonzero(y != yhat)[0]
            
            if len(misclassified) == 0:
                return
            
            self.iters += 1
            if self.iters > 1000:
                print('PLA cannot converge.')
                return
            
            i = np.random.choice(misclassified)
            self.w = self.w + y[i]*X[i,:]

In [6]:
total_iters = 0

for i in range(1000):
    
    ds = Dataset(10)
    lr = LinearRegression()
    lr.fit(ds.X, ds.y)

    pla = PLA()
    pla.fit(ds.X, ds.y, initial_w=lr.w)
    total_iters += pla.iters

total_iters / 1000

4.273

# Non-linear transformation

In [67]:
class Dataset_non_linear:
    def __init__(self, N):
        self.X = np.random.uniform(-1, 1, (N, 3))
        self.X[:, 2] = 1
        self.y = np.sign(self.X[:, 0]**2 + self.X[:, 1]**2 - 0.6)
        i_flip = np.arange(N)
        np.random.shuffle(i_flip)
        i_flip = i_flip[:(N//10)]
        self.y[i_flip] *= -1

In [69]:
total_Ein = 0
for i in range(1000):
    ds = Dataset_non_linear(100)
    lr = LinearRegression()
    lr.fit(ds.X, ds.y)
    yhat = lr.predict(ds.X)
    
    total_Ein += np.mean(yhat != ds.y)
    
total_Ein / 1000

0.44049000000000005

In [74]:
def transform(X):
    X_t = np.zeros((len(X), 6))
    X_t[:, 0] = 1
    X_t[:, 1] = X[:, 0]
    X_t[:, 2] = X[:, 1]
    X_t[:, 3] = X[:, 0] * X[:, 1]
    X_t[:, 4] = X[:, 0]**2
    X_t[:, 5] = X[:, 1]**2
    return X_t

In [81]:
total_weights = 0
ds = Dataset_non_linear(1000)
ds.X = transform(ds.X)
lr = LinearRegression()
lr.fit(ds.X, ds.y)
    
total_weights += lr.w
    
total_weights

array([-0.89785694,  0.02661686, -0.03747541,  0.02564458,  1.49713671,
        1.3593845 ])